In [27]:
import pandas as pd
import numpy as np
from pulp import *
from variables import *


In [28]:
def make_prob():
    prob = LpProblem('BRS',LpMinimize)
    x1 = LpVariable('x1', lowBound = 0, cat = 'Integer')
    x2 = LpVariable('x2', lowBound = 0, cat = 'Integer')
    x3 = LpVariable('x3', lowBound = 0, cat = 'Integer')
    x4 = LpVariable('x4', lowBound = 0, cat = 'Integer')
    
    return prob, x1, x2, x3, x4


In [29]:
Cost = pd.read_excel(filename_truck_cost, sheet_cost).values
Truck = pd.read_excel(filename_truck_cost, sheet_truck_cap)
t=Truck.columns[1:].values
truck=Truck.values
print(t)
tvol = truck[1,1:]
tweight = truck[2,1:]
cost = dict()
for r in Cost:
    br = r[0].split(' + ')
    b=''
    for j in br:
        b+= j.split(' ')[0] + ' + '
    b=b[:-3]
    print(b)
    cost[b]=r[1:5]

[407 709 1109 'Dummy']
AKBARPUR + SHAHGANJ
ANANDNAGAR + NAUTANWA
ATRAULIA + AZAMGARH
BALLIA
BASTI + KHALILABAD
BELTHRA + BARHALGANJ
CHANDAULI
DEORIA
DUMARIYAGANJ + BANSI
GHAZIPUR
GPG + MZP
JANUNPUR
KND
KRO
LALGANJ
MAU
MBP
PADRAUNA + SISWA
ROBERTSGANJ + RENUKOOT
SRT


In [30]:
sheet = pd.read_excel(output_pivot_table)
data = sheet.values
print(data)
n=len(data)
m=len(data[0])


[[' ' '12/1/2019' '12/1/2019' ... '12/30/2019' '12/31/2019' '12/31/2019']
 ['Location' 'weight(kg)' 'volume(m3)' ... 'volume(m3)' 'weight(kg)'
  'volume(m3)']
 ['Padrauna + Siswa Branch' 0 0 ... 24.54818578707 2633.396341050001
  23.14011413336]
 ...
 ['Anandnagar + Nautanwa Branch' 0 0 ... 16.68797964927 2505.206812267
  20.588565450352]
 ['Ballia Branch' 0 0 ... 0 2148.136816235 11.659237982]
 ['Chandauli Branch' 0 0 ... 0 2012.85839716 12.9024413953]]


In [32]:
writer = pd.ExcelWriter(optimizer_output, engine='openpyxl')

lis = ['Branch',t[0],t[1],t[2],t[3],'Cost','VU']

branches = ['Branch']
for i in range(2,n):
    branches.append(data[i][0])

cc = ['Monthly Cost']
vv = ['Average Vehicle Utilisation']

for i in range(1,len(branches)):
    cc.append(0)
    vv.append(0)

cost_dw = [branches,cc]        
vu_dw = [branches,vv]

for j in range(1,m,2):
    date = data[0][j].split('/')
    date = date[0]+'-'+date[1]+'-'+date[2]
    print(date)
    table = []
    table.append(lis)
    cost_list = [date]
    vu_list = [date]
    
    for i in range(2,n):
        br=data[i][0].upper().split(' + ')
        b=''
        for k in br:
            b+= k.split(' ')[0] + ' + '
        b=b[:-3]
        weight = data[i][j]
        vol = data[i][j+1]
        c=cost[b]
        
        prob, x1, x2, x3, x4 = make_prob()
        
        prob+= c[0]*x1 + c[1]*x2 + c[2]*x3 + c[3]*x4
        prob+= tweight[0]*x1 + tweight[1]*x2 + tweight[2]*x3 + tweight[3]*x4 >= weight
        prob+= tvol[0]*x1 + tvol[1]*x2 + tvol[2]*x3 + tvol[3]*x4 >= vol
        
        status = prob.solve()
        
        x = [value(x1),value(x2),value(x3),value(x4)]
        for i in range(4):
            if x[i]==None:
                x[i]=0
        
        if weight == 0:
            vu = 0
        else:
            vuw = weight/(tweight[0]*x[0] + tweight[1]*x[1] + tweight[2]*x[2] + tweight[3]*x[3])
            vuv = vol/(tvol[0]*x[0] + tvol[1]*x[1] + tvol[2]*x[2] + tvol[3]*x[3])
            vu = max(vuw,vuv)
        
        cost_list.append(value(prob.objective))
        vu_list.append(vu*100)
        
        table.append([b+' Branch',x[0],x[1],x[2],x[3],value(prob.objective),vu*100])
    
    cost_dw.append(cost_list)
    vu_dw.append(vu_list) 
    
    df = pd.DataFrame(table)
    df.to_excel(writer, date, index=False)


cost_dw = np.array(cost_dw).T
vu_dw = np.array(vu_dw).T

for i in range(1,len(cost_dw)):
    sumc  = 0
    for j in cost_dw[i][2:]:
        j = j.split('.')
        if len(j)>1:
            k = int(j[0])+pow(10,-1*len(j[1]))*int(j[1])
        else:
            k=int(j[0])
        sumc+= k
    
    cost_dw[i][1] = sumc
    
    c = 0
    s = 0
    for j in vu_dw[i][2:]:
        j = j.split('.')
        if len(j)>1:
            k = int(j[0])+pow(10,-1*len(j[1]))*int(j[1])
        else:
            k=int(j[0])
        if k!=0:
            c+=1
            s+=k
    try:
        vu_dw[i][1]=s/c
    except:
        a=1
            

cost_df = pd.DataFrame(cost_dw)
vu_df = pd.DataFrame(vu_dw)

cost_df.to_excel(writer, 'BRS cost Daywise', index=False)
vu_df.to_excel(writer, 'Vehicle Utilisation Daywise', index=False)

    

writer.save()
    
        
        
        
        
        
        
        
        
        

12-1-2019
12-2-2019
12-3-2019
12-4-2019
12-5-2019
12-6-2019
12-7-2019
12-8-2019
12-9-2019
12-10-2019
12-11-2019
12-12-2019
12-13-2019
12-14-2019
12-15-2019
12-16-2019
12-17-2019
12-18-2019
12-19-2019
12-20-2019
12-21-2019
12-22-2019
12-23-2019
12-24-2019
12-25-2019
12-26-2019
12-27-2019
12-28-2019
12-29-2019
12-30-2019
12-31-2019
